# Number of secondaries
Notebook taking a look at the number and energy of secondary electrons trapped primary ($\beta$-decay) electrons produce.

S. Jones (25/09/25)

In [21]:
# Basic imports and settings
import matplotlib.pyplot as plt
import matplotlib as mpl
import numpy as np
import scipy.constants as const
from scipy.integrate import quad
from scipy.interpolate import interp1d
from scipy.optimize import curve_fit
import lmfit
import time

In [22]:
# Import the cross-section classes
from src.Rudd1991 import RuddXSec
from src.Mott import MottXSec
import src.Constants as myconst

In [23]:
# Make matplotlib figures better and allow latex
plt.rcParams['text.usetex'] = True
# Make figures bigger
plt.rcParams['figure.figsize'] = (8, 5)
# Make font sizes bigger
plt.rcParams.update({'font.size': 14})

Set up a function which, for a given energy, start pitch angle and trapping criterion gives the total number and energies of secondaries produced.

In [24]:
def CalcCDF_Rudd_SDCS_W(T):
    """
    Calculate the cumulative distribution function for the singly-differential 
    ionisation cross-section in W. 

    Parameters:
    -----------
    T : (float) The kinetic energy of the electron in eV.
    """
    # First, the values of W to be probed
    diffArr = np.logspace(-3, np.log10(T / 2.0), 100)
    WArr = (T - myconst.IONIZATIONENERGYH) * np.ones_like(diffArr) - diffArr

    # Calculate the cross-sections
    sdcsW = RuddXSec(T).SinglyDifferentialXSec_W(WArr)

    # Add a zero point to the end of the arrays
    WArr = np.append(WArr, 0.0)
    sdcsW = np.append(sdcsW, 0.0)
    # Reverse arrays
    sdcsW = np.flip(sdcsW)
    WArr = np.flip(WArr)

    # Get the normalisation constant
    sdcsWIntegral = np.trapz(sdcsW, x=WArr)

    # Create an array to store the CDF
    cdf = np.zeros_like(WArr)
    for i, W in enumerate(WArr):
        cdf[i] = np.trapz(sdcsW[:i], x=WArr[:i])

    cdf /= cdf[-1]

    return WArr, cdf

def CalcCDF_DDCS_Theta(T, W):
    """
    Calculate the CDF for the doubly-differential cross-section in theta for a
    given value of W and T.

    Parameters
    ----------
    T: Incident energy in eV
    W: Energy transfer in eV
    """
    
    nPnts = 100
    theta = np.logspace(-5, np.log10(np.pi / 2.0), nPnts)
    ddcsTheta = np.zeros_like(theta)
    for i in range(1, nPnts):
        ddcsTheta[i] = RuddXSec(T).DoublyDifferentialXSec(W, theta[i])

    cdf = np.zeros_like(theta)
    for i, th in enumerate(theta):
        cdf[i] = np.trapz(ddcsTheta[:i], x=theta[:i])

    cdf /= cdf[-1]

    return theta, cdf

def RotateToCoords(v, xAx, yAx, zAx):
    """
    Rotate a vector to a new coordinate system

    Parameters
    ----------
    v: Unit vector to be rotated
    """
    x1Prime = (xAx * v[0])[0] + (yAx * v[1])[0] + (zAx * v[2])[0]
    x2Prime = (xAx * v[0])[1] + (yAx * v[1])[1] + (zAx * v[2])[1]
    x3Prime = (xAx * v[0])[2] + (yAx * v[1])[2] + (zAx * v[2])[2]
    return np.array([x1Prime, x2Prime, x3Prime])

def GetScatteredVector(vIn, theta):
    """
    Calculation of the scattered velocity vector

    Parameters
    ----------
    vIn: Incident velocity vector
    theta: Scattering angle in radians
    """

    phiGen = 2.0 * np.pi * np.random.rand()

    # Original direction in global coordinates
    originalDir = vIn / np.linalg.norm(vIn)
    # Original direction in scattering frame
    oldDir = np.array([0.0, 0.0, 1.0])
    # New direction in scattering frame
    newDir = np.array([np.sin(theta) * np.cos(phiGen), 
                       np.sin(theta) * np.sin(phiGen), 
                       np.cos(theta)])

    # Define other axes
    ax2 = np.array([-originalDir[1] / originalDir[0], 1.0, 0.0])  
    ax2 /= np.linalg.norm(ax2)
    ax3 = np.cross(ax2, originalDir)
    ax3 /= np.linalg.norm(ax3)  

    return RotateToCoords(newDir, ax2, ax3, originalDir)

# Now create code for a CDF of the Mott scattering cross-section
def CalcCDF_Mott_Theta(T):
    """
    Calculate the CDF for the Mott scattering cross-section in theta for a given
    value of T.

    Parameters
    ----------
    T: Incident energy in eV
    """
    nPnts = 300
    thetaArr = np.logspace(-5, np.log10(np.pi * 0.99999), nPnts)
    mott = MottXSec(T)
    sdcsTheta = mott.SinglyDifferentialXSec_theta(thetaArr)

    cdf = np.zeros_like(thetaArr)
    for i, th in enumerate(thetaArr):
        cdf[i] = np.trapz(sdcsTheta[:i], x=thetaArr[:i])

    cdf /= cdf[-1]

    return thetaArr, cdf

In [25]:
TATOMDENS = 1e18 # Tritium atom number density in m^-3
def PropagateElectron(T: float, initialPitchAngle: float, minPitchAngle: float):
  """
  Propagate an electron with a given energy and pitch angle until it escapes 
  the trap.

  Parameters
  ----------
  T: (float) The initial kinetic energy of the electron in eV
  initialPitchAngle: (float) The initial pitch angle of the electron in radians
  minPitchAngle: (float) The minimum pitch angle which is trapped in radians

  Returns
  -------
  nSecondaries: (int) The number of secondary electrons which are trapped
  eEnergies: (list) The energies of the secondary electrons in eV
  """

  trappedTime = 0.0

  nSecondaries = 0
  eEnergies = []

  pitchAngle = initialPitchAngle
  ke = T

  while abs(pitchAngle) > minPitchAngle:
    gamma = 1.0 + ke * const.e / (const.m_e * const.c**2)
    beta = np.sqrt(1.0 - 1.0 / gamma**2)
    speed = beta * const.c
    # Generate a random cyclotron phase
    cyclotronPhase = 2.0 * np.pi * np.random.rand()
    # Calculate the velocity vector
    velocity = speed * np.array([np.sin(pitchAngle) * np.cos(cyclotronPhase),
                                np.sin(pitchAngle) * np.sin(cyclotronPhase),
                                np.cos(pitchAngle)])
    # Get cross-sections
    rudd = RuddXSec(ke)
    mott = MottXSec(ke)
    totalXSec = rudd.TotalXSec() + mott.TotalXSec()
    # Get the mean free path
    meanFreePath = 1.0 / (TATOMDENS * totalXSec)
    # Get the time to the next ionisation event
    trappedTime += np.random.exponential(meanFreePath / speed)

    # Determine if this is a elastic or ionisation event
    if np.random.rand() < mott.TotalXSec() / totalXSec:
      # Elastic scattering event
      # Get the new pitch angle
      thetaArr, cdfMottTheta = CalcCDF_Mott_Theta(ke)
      scatteringAngle = np.interp(np.random.rand(), cdfMottTheta, thetaArr)
      # Get the new velocity vector direction
      vDir = GetScatteredVector(velocity, scatteringAngle)
      # Update the pitch angle
      pitchAngle = np.arctan(np.sqrt(vDir[0]**2 + vDir[1]**2) / vDir[2])

    else:
      # Ionisation event
      nSecondaries += 1
      # Get the energy of the secondary electron
      wArr, cdfRuddW = CalcCDF_Rudd_SDCS_W(ke)
      sampledW = np.interp(np.random.rand(), cdfRuddW, wArr)

      # and the energy of the other outgoing electron
      eEnergies.append(ke - sampledW - myconst.IONIZATIONENERGYH)

      # Now get the scattering angle
      thetaRuddArr, cdfRuddTheta = CalcCDF_DDCS_Theta(ke, sampledW)
      scatteringAngle = np.interp(np.random.rand(), cdfRuddTheta, 
                                  thetaRuddArr)
      # Get the velocity vector of the secondary electron
      vDir = GetScatteredVector(velocity, scatteringAngle)
      pitchAngle = np.arctan(np.sqrt(vDir[0]**2 + vDir[1]**2) / vDir[2])
      ke = sampledW

  return nSecondaries, eEnergies

In [ ]:
n, eList = PropagateElectron(18.575e3, 89.0 * np.pi / 180., 85. * np.pi / 180.)
# Plot the energies as a histogram
plt.hist(eList, bins=5)
plt.ylabel("Number of secondary electrons")
plt.xlabel("Energy [eV]")
plt.show()

In [ ]:
# Repeat this for multiple iterations with different pitch angles
initialKE = 18.575e3
minPitchAngle = 85.0 * np.pi / 180.0
nGenerated = 0
NELECTRONS = 400
allEnergies = []

while nGenerated < NELECTRONS:
  # Generate a random direction
  theta = np.arccos(2 * np.random.rand() - 1)
  phi = 2 * np.pi * np.random.rand()
  x = np.sin(theta) * np.cos(phi)
  y = np.sin(theta) * np.sin(phi)
  z = np.cos(theta)
  initialPitchAngle = np.arctan(np.sqrt(x**2 + y**2) / z)
  if abs(initialPitchAngle) >= minPitchAngle:
    nGenerated += 1

    _, eList = PropagateElectron(initialKE, initialPitchAngle, minPitchAngle)
    allEnergies.extend(eList)

plt.hist(allEnergies, bins='auto', density=True)
plt.ylabel("Probability")
plt.xlabel("Energy [eV]")
plt.title("Primary energy = 18.575 keV")
plt.xlim([0, 100])
plt.show()